# Base de données: interroger une base de données MySQL

## Imports

In [ ]:
import os
import sqlite3
import zipfile
from pathlib import Path
import requests

# Créer un dossier data s'il n'existe pas encore
if not os.path.exists("../data"):
    os.mkdir("../data")

## Créer une base de données SQLite

In [ ]:
# Créer un répertoire DB s'il n'existe pas encore
db_path = '../data/db'
if not os.path.exists(db_path):
    os.mkdir(db_path)

In [ ]:
# Récupérer le fichier ZIP qui contient la DB de test
url = "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"
filename = url.split("/")[-1]
# Récupérer le fichier zip dans la RAM
response = requests.get(url)

In [ ]:
# Ecrire le fichier sur le disque
with open(os.path.join(db_path, filename), 'wb') as f:
    f.write(response.content)

In [ ]:
# Extraire le zip pour obtenir la db
with zipfile.ZipFile(os.path.join(db_path, filename), 'r') as zip_ref:
    zip_ref.extractall(db_path)

## Schéma de la base de données

<img src="https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg" />

## Se connecter à la base de données

In [ ]:
conn = sqlite3.connect(os.path.join(db_path, "chinook.db"))
c = conn.cursor()

## Trouver l'identifiant de Miles Davis

In [ ]:
artist = 'Miles Davis'

query1 = f"""
    SELECT
        artistId
    FROM
        artists
    WHERE
        name = '{artist}';
"""
c.execute(query1)
artist_id = c.fetchone()[0]

print(artist_id)

## Trouver les identifiants d'albums de Miles Davis

In [ ]:
query2 = f"""
    SELECT
        albumId
    FROM 
        albums 
    WHERE 
        artistId = '{artist_id}';
"""
albums_ids = []
for row in c.execute(query2):
    albums_ids.append(str(row[0]))

print(albums_ids)

## Trouver les morceaux des albums de Miles Davis

In [ ]:
query3 = f"""
    SELECT
        name,
        albumId
    FROM
        tracks
    WHERE 
        albumId IN ({",".join(albums_ids)});
"""

songs = set()
for row in c.execute(query3):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)

## En une seule requête...

In [ ]:
query4 = f"""
    SELECT
        tracks.name,
        tracks.albumId
    FROM
        artists
        JOIN albums ON artists.artistId = albums.artistId
        JOIN tracks ON albums.albumId = tracks.albumId
    WHERE 
        artists.name = '{artist}';
"""

songs = set()
for row in c.execute(query4):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)

## Pour en savoir plus

- Tutoriel SQLite : https://www.sqlitetutorial.net/